In [26]:
import fairseq

In [ ]:
#     --tgtdict /workspace/diffs.fairseq.vocab --tgtdict /workspace/messages.fairseq.vocab \

    --srcdict /workspace/fairseq.vocab --tgtdict /workspace/fairseq.vocab \

In [73]:
!fairseq-preprocess \
    --source-lang diff --target-lang msg \
    --trainpref /workspace/tmp/dataset_10k.bpe/prepared/train \
    --validpref /workspace/tmp/dataset_10k.bpe/prepared/valid \
    --testpref /workspace/tmp/dataset_10k.bpe/prepared/test \
    --destdir /workspace/tmp/data-bin/dataset_10k.bpe.diff-msg \
    --nwordssrc 16000 --nwordstgt 8000 \
    --tokenizer space \
    --bpe sentencepiece \
    --workers 60

Namespace(alignfile=None, bpe='sentencepiece', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/workspace/tmp/data-bin/dataset_10k.bpe.diff-msg', fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=16000, nwordstgt=8000, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='diff', srcdict=None, target_lang='msg', task='translation', tbmf_wrapper=False, tensorboard_logdir='', testpref='/workspace/tmp/dataset_10k.bpe/prepared/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer='space', trainpref='/workspace/tmp/dataset_10k.bpe/prepared/train', user_dir=None, validpref='/workspace/tmp/dataset_10k.bpe/prepared/valid', workers=60)
| [diff] Dictionary: 13175 types
| [diff] /workspace/tmp/dataset_10k.bpe/prepared/train.

In [57]:
!fairseq-preprocess --source-lang de --target-lang en \
    --trainpref /workspace/iwslt14.tokenized.de-en/train --validpref /workspace/iwslt14.tokenized.de-en/valid --testpref /workspace/iwslt14.tokenized.de-en/test \
    --destdir /workspace/tmp/data-bin/iwslt14.tokenized.de-en \
    --workers 20

Namespace(alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/workspace/tmp/data-bin/iwslt14.tokenized.de-en', fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='de', srcdict=None, target_lang='en', task='translation', tbmf_wrapper=False, tensorboard_logdir='', testpref='/workspace/iwslt14.tokenized.de-en/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='/workspace/iwslt14.tokenized.de-en/train', user_dir=None, validpref='/workspace/iwslt14.tokenized.de-en/valid', workers=20)
| [de] Dictionary: 8847 types
| [de] /workspace/iwslt14.tokenized.de-en/train.de: 160239 sents, 4035591 tokens, 0.0% replaced 

In [33]:
!fairseq-train \
    /workspace/tmp/data-bin/dataset_10k_diff_msg \
    --save-dir /workspace/tmp/models/dataset_10k_diff_msg \
    --tensorboard-logdir /workspace/tmp/tb_log \
    --arch transformer_iwslt_de_en \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-3 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096

Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_iwslt_de_en', attention_dropout=0.0, best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='/workspace/tmp/data-bin/dataset_10k_diff_msg', dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=4, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_input_dim=512, decoder_layers=6, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.3, encoder_attention_heads=4, encoder_embed_dim=512, encoder_embed_path=None, encoder_ffn_embed_dim=1024,

| loaded 7604 examples from: /workspace/tmp/data-bin/dataset_10k_diff_msg/train.diff-msg.diff
| loaded 7604 examples from: /workspace/tmp/data-bin/dataset_10k_diff_msg/train.diff-msg.msg
| /workspace/tmp/data-bin/dataset_10k_diff_msg train diff-msg 7604 examples
| epoch 001:  51%|▌| 359/705 [14:20<14:44,  2.56s/it, loss=10.382, nll_loss=10.059, ppl=1066.41, wps=47, ups=0, wpb=113.131, bsz=11.128, num_updates=359, lr=0.00044875, gnorm=3.725, clip=0.000, oom=0.000, wall=861, train_wall=855]^C


In [6]:
from fairseq.models.transformer import TransformerModel, transformer_iwslt_de_en

In [50]:
model = TransformerModel.from_pretrained(
    model_name_or_path="/workspace/tmp/models/dataset_10k_diff_msg", 
    checkpoint_file="checkpoint_best.pt", 
    data_name_or_path="../../data-bin/dataset_10k_raw_diff_msg")

loading archive file /workspace/tmp/models/dataset_10k_diff_msg
| [diff] dictionary: 16000 types
| [msg] dictionary: 7264 types
Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_iwslt_de_en', attention_dropout=0.0, best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='/workspace/tmp/data-bin/dataset_10k_raw_diff_msg', dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=4, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_input_dim=512, decoder_layers=6, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distribut

In [53]:
model.translate("""2234""")

'add'

In [8]:
!ls /workspace/tmp/data-bin/dataset_10k_diff_msg/

dict.diff.txt		 train.diff-msg.msg.bin   valid.diff-msg.msg.bin
dict.msg.txt		 train.diff-msg.msg.idx   valid.diff-msg.msg.idx
train.diff-msg.diff.bin  valid.diff-msg.diff.bin
train.diff-msg.diff.idx  valid.diff-msg.diff.idx


In [12]:
import os
os.stat("/workspace/tmp/data-bin/dataset_10k_diff_msg/dict.diff.txt")

os.stat_result(st_mode=33188, st_ino=11376515, st_dev=77, st_nlink=1, st_uid=0, st_gid=0, st_size=104200, st_atime=1575330923, st_mtime=1575330922, st_ctime=1575330922)

In [54]:
!fairseq-generate \
    /workspace/tmp/data-bin/dataset_10k_raw_diff_msg \
    --path /workspace/tmp/models/dataset_10k_diff_msg/checkpoint_best.pt \
    --beam 5 --remove-bpe

Namespace(beam=5, bpe=None, cpu=False, criterion='cross_entropy', data='/workspace/tmp/data-bin/dataset_10k_raw_diff_msg', dataset_impl=None, diverse_beam_groups=-1, diverse_beam_strength=0.5, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, gen_subset='test', lazy_load=False, left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, lr_scheduler='fixed', lr_shrink=0.1, match_source_len=False, max_len_a=0, max_len_b=200, max_sentences=None, max_source_positions=1024, max_target_positions=1024, max_tokens=12000, memory_efficient_fp16=False, min_len=1, min_loss_scale=0.0001, model_overrides='{}', momentum=0.99, nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, no_repeat_ngram_size=0, num_shards=1, num_workers=1, optimizer='nag', path='/workspace/tmp/models/dataset_10k_diff_msg/checkpoint_best.pt', prefix_size=0, print_alignment=False, quiet=False, raw_text=False, remove_bpe=

S-411	<file> server.py <chunk> class server(hashable): <nl> <unk> = roles <nl> member = <unk> <nl> <unk> = self <nl> <del> <nl> <del> if member.id == <unk> <nl> <del> <unk> = member <nl> <del> <nl> <unk> <nl> <nl> <add> if <unk> in guild: <nl> <add> <unk> = <unk> <nl> <add> <nl> for presence in <unk> []): <nl> user_id = <unk> <nl> member = <unk> <nl>
T-411	support upcoming guild <<unk>> <<unk>> change
H-411	-1.857202410697937	add
P-411	-1.8195 -1.8949
S-91	<file> changelog.txt <chunk> changelog <nl> <unk> (unreleased) <nl> ------------------ <nl> <nl> <add> * fixed <unk> filter breaking when there is no image <nl> <nl> <nl> <unk> <unk> <nl> <file> <unk> <chunk> <unk> = <unk> <nl> <nl> <nl> def <unk> <nl> <del> if not <unk> <nl> <add> if not image or not <unk> <nl> return <unk> <unk> <nl> <nl> x, y = <unk> <nl>
T-91	fixed <<unk>> filter breaking when there is no image
H-91	-1.8571560382843018	add
P-91	-1.8195 -1.8948
S-16	<file> 0001_initial.py <chunk> <nl> <del> # <unk> <unk> <nl> <add

H-419	-1.857205867767334	add
P-419	-1.8195 -1.8949
S-165	<file> <unk> <chunk> <nl> <add> {% extends "shop/checkout/dialog-base.html" %} <nl> <add> {% block dialog_form %} <nl> <form name="{{ <unk> }}" novalidate> <nl> <fieldset> <nl> <legend>{{ customer_form.legend }}</legend> <nl> {{ <unk> }} <nl> </fieldset> <nl> <del> </form> <nl> \ no newline at end of file <nl> <add> </form> <nl> <add> {% endblock %} <nl>
T-165	fixed forgotten refactoring of customerform to use a base dialog form
H-165	-1.857142686843872	add
P-165	-1.8194 -1.8948
S-447	<file> checkout.py <chunk> class <unk> <nl> addresses = <unk> <nl> addresses = <unk> <nl> <unk> = <unk> <unk> <nl> <del> <unk> ' – ')) for <unk> in <unk> <nl> <add> <unk> <unk> <unk> ' – <unk> <nl> <add> for number, <unk> in <unk> 1)] <nl> <unk> <unk> <nl> else: <nl> <unk> <nl>
T-447	prefix address choices with numbers
H-447	-1.8571782112121582	add
P-447	-1.8195 -1.8949
S-201	<file> django.po <chunk> msgstr "" <nl> "project-id-version: <unk> <nl> "r

S-368	<file> <unk> <chunk> <nl> <del> <form <unk> name="{{ <unk> }}" novalidate> <nl> <del> {{ <unk> }} <nl> <del> </form> <nl> <del> {% if <unk> %} <nl> <del> {% include <unk> %} <nl> <del> {% endif %} <nl> \ no newline at end of file <nl>
T-368	use this form from the default templates
H-368	-1.8585262298583984	add
P-368	-1.8212 -1.8959
S-309	<file> <unk> <chunk> class flexx: <nl> <unk> <nl> def <unk> <unk> <nl> msg = <unk> <nl> <del> if <unk> <nl> <add> if <unk> and <unk> # <unk> can be none for syntax err <nl> stack = <unk> <nl> if <unk> in <unk> <nl> <unk> <nl>
T-309	tiny fix for js error handler when syntax error
H-309	-1.8571922779083252	add
P-309	-1.8195 -1.8949
S-161	<file> address.py <chunk> class addresseditview(genericapiview): <nl> form = <unk> cart=cart) <nl> if form.is_valid(): <nl> return <unk> <nl> <del> return <unk> <nl> <add> <unk> = <unk> <unk> <nl> <add> return <unk> <unk> <nl> <nl> def delete(self, request, <unk> *args, **kwargs): <nl> if priority != 'add': <nl>
T-

S-377	<file> conf.py <chunk> sys.path.insert(0, <unk> <nl> os.environ['django_settings_module'] = <unk> <nl> <unk> = <unk> <nl> <nl> <del> <unk> <nl> from django.conf import settings <nl> <del> import django <nl> <del> django.setup() <nl> <nl> # -- general configuration <unk> <nl> <nl>
T-377	resolve double django setup
H-377	-1.8584299087524414	add
P-377	-1.8211 -1.8958
S-290	<file> role.py <chunk> dealings in the software. <nl> import asyncio <nl> <nl> from .permissions import permissions <nl> <add> from .errors import invalidargument <nl> from .colour import colour <nl> from .mixins import <unk> <nl> from .utils import <unk> <nl>
T-290	fix nameerror in roleedit when moving roles
H-290	-1.8572304248809814	add
P-290	-1.8195 -1.8949
S-194	<file> authentication.md <chunk> http signature (currently a [ietf draft][http-signature-ietf-draft]) provides a <nl> <unk> <url> <nl> <unk> permissions.md <nl> [throttling]: throttling.md <nl> <del> [csrf-ajax]: <url> <nl> <add> [csrf-ajax]: <url> <nl

...

S-324	<file> funding.md <chunk> rest framework continues to be <unk> and <unk> <unk> but we <unk> <nl> * the <unk> and <unk> <unk> including schema generation for both <unk> and <unk> a python client library, a command line client, and addressing of a large number of outstanding issues. <nl> * the <unk> release, including javascript client library, and api documentation, complete with <unk> code <unk> <nl> * tom <unk> the <unk> of django rest framework, working on the project <unk> <nl> <del> * over <unk> closed issues on <unk> since tom christie started working on the project full-time <unk> 2016 - march <unk> <nl> <add> * around <unk> issues and pull requests closed per month since tom christie started working on the project <unk> <nl> * a community & operations manager position <unk> for 4 <unk> helping <unk> the business and <unk> <unk> <nl> * <unk> development time for the work on the javascript client library and api documentation <unk> <nl> <nl> <chunk> rest framework continues 

S-346	<file> <unk> <chunk> <nl> <add> """ <nl> <add> test that importing flexx <unk> does not pull in any more flexx <nl> <add> <unk> than strictly necessary, and not any more <unk> party <nl> <add> dependencies than <unk> <nl> <add> """ <nl> <add> <nl> <add> import os <nl> <add> import sys <nl> <add> import subprocess <nl> <add> <nl> <add> from pytest import raises <nl> <add> <nl> <add> import flexx <nl> <add> <nl> <add> <nl> <add> # minimum that will be imported when importing flexx <nl> <add> <unk> = flexx <nl> <add> <unk> = <unk> <nl> <add> <unk> = <unk> <nl> <add> <nl> <add> ## generic code <nl> <add> <nl> <add> def <unk> <unk> <unk> <nl> <add> """ import the given module in subprocess and return loaded modules <nl> <add> <nl> <add> import a certain module in a clean subprocess and return the <nl> <add> projects modules that are <unk> loaded. the given depth <nl> <add> indicates the module level (i.e. <unk> will only yield <unk> <nl> <add> but not <unk> <nl> <add> """ <nl> <add> <

S-26	<file> delivery.py <chunk> from <unk> import <unk> <nl> from shop.models.order import orderitemmodel <nl> from <unk> import <unk> <nl> from shop.modifiers.pool import cart_modifiers_pool <nl> <add> from <unk> import <unk> <nl> from <unk> import orderdetailserializer <nl> <nl> <nl> <chunk> class <unk> <nl> context = {'request': request, 'render_label': <unk> <nl> customer_serializer = <unk> <nl> order_serializer = <unk> context=context) <nl> <add> <unk> = <unk> context=context) <nl> content = <unk> <nl> 'customer': <unk> <nl> <del> 'data': order_serializer.data, <nl> <del> <unk> <unk> <nl> <add> <unk> order_serializer.data, <nl> <add> <unk> <unk> <nl> }) <nl> return <unk> <nl> <nl> <file> delivery.py <chunk> <nl> <add> # -*- coding: utf-8 -*- <nl> <add> from __future__ import unicode_literals <nl> <add> <nl> <add> from rest_framework import serializers <nl> <add> from shop.conf import app_settings <nl> <add> from <unk> import <unk> <unk> <nl> <add> from shop.modifiers.pool import c

 89%|████████████████████████████████▉    | 8/9 [00:29<00:03,  3.55s/it, wps=34]S-401	<file> contributing.rst <chunk> <nl> <add> ============ <nl> <add> contributing <nl> <add> ============ <nl> <add> <nl> <add> setup <nl> <add> ===== <nl> <add> <nl> <add> fork <unk> repository on `github <<url>>`_ and follow these steps: <nl> <add> <nl> <add> * create a virtualenv and activate it <nl> <add> * clone your repository locally <nl> <add> * cd into the repository and type `pip install -r <unk> (this will install both optional and base <unk> useful during <unk> <nl> <add> <nl> <add> issues <nl> <add> ====== <nl> <add> <nl> <add> you can find the list of <unk> enhancements and feature requests on the <nl> <add> `issue tracker <<url>>`_. if you want to fix an issue, pick up one and <nl> <add> add a comment <unk> you're working on it. if the resolution implies a discussion or if you <unk> the comments on the <nl> <add> issue are growing pretty fast, move the discussion to the <unk> group <<url>

S-241	<file> <unk> <chunk> <nl> # python <unk> <nl> <nl> <unk> is a pure python interface to the raspberry pi camera module for python 2.7 (or above) or python 3.2 (or above). the library is written and maintained by <unk> <unk> <nl> <add> <nl> <add> also see the [camera <unk> page. <nl> <add> <nl> <add> ## installation <nl> <add> <nl> <add> the <unk> library is available in the raspbian <unk> install with `apt`: <nl> <add> <nl> <add> ``` <nl> <add> sudo apt-get update <nl> <add> sudo apt-get install <unk> <nl> <add> ``` <nl> <add> <nl> <add> alternatively, the python3 package is installed with `apt-get install <unk> also the documentation is available with `apt-get install <unk> <nl> <add> <nl> <add> ## usage <nl> <add> <nl> <add> first, in the python prompt or at the top of a python file, <unk> <nl> <add> <nl> <add> ``` <nl> <add> import <unk> <nl> <add> ``` <nl> <add> <nl> <add> this will make the library available to the script. <nl> <add> <nl> <add> now create an instance of the <

In [68]:
toks = "▁ <file> ▁__ init __. py ▁ <chunk> ▁from ▁client ▁import ▁client testcase ▁ <nl> ▁from ▁payment ▁import ▁pa yon delivery testcase , ▁general payment backend testcase ▁ <nl> ▁from ▁util ▁import ▁currency field testcase , ▁cart util stestcase ▁ <nl> ▁from ▁shipping ▁import ▁general ship ping backend testcase ▁ <nl> ▁ <del> ▁from ▁product ▁import ▁product testcase ▁ <nl> ▁\ ▁no ▁newline ▁at ▁end ▁of ▁file ▁ <nl> ▁ <add> ▁from ▁product ▁import ▁product testcase ▁ <nl> ▁ <add> ▁from ▁views ▁import ▁product detail view testcase ▁ <nl> ▁\ ▁no ▁newline ▁at ▁end ▁of ▁file ▁ <nl> ▁ <file> ▁views . py ▁ <chunk> ▁ <nl> ▁ <add> ▁# - *- ▁coding : ▁utf -8 ▁-*- ▁ <nl> ▁ <add> ▁from ▁decimal ▁import ▁decimal ▁ <nl> ▁ <add> ▁from ▁sho p . models . product model ▁import ▁product ▁ <nl> ▁ <add> ▁from ▁sho p . views . product ▁import ▁product detail view ▁ <nl> ▁ <add> ▁from ▁unittest ▁import ▁testcase ▁ <nl> ▁ <add> ▁ <nl> ▁ <add> ▁ <nl> ▁ <add> ▁ <nl> ▁ <add> ▁class ▁product detail view testcase ( testcase ): ▁ <nl> ▁ <add> ▁def ▁setup ( self ): ▁ <nl> ▁ <add> ▁ <nl> ▁ <add> ▁self . product ▁= ▁product () ▁ <nl> ▁ <add> ▁self . product . name ▁= ▁' test ' ▁ <nl> ▁ <add> ▁self . product . short _ description ▁= ▁' test ' ▁ <nl> ▁ <add> ▁self . product . long _ description ▁= ▁' test ' ▁ <nl> ▁ <add> ▁self . product . unit _ price ▁= ▁decimal (' 1.0 ') ▁ <nl> ▁ <add> ▁self . product . save () ▁ <nl> ▁ <add> ▁ <nl> ▁ <add> ▁self . view ▁= ▁product detail view ( kwargs ={' pk ': self . product . id }) ▁ <nl> ▁ <add> ▁ <nl> ▁ <add> ▁def ▁teardown ( self ): ▁ <nl> ▁ <add> ▁self . product . delete () ▁ <nl> ▁ <add> ▁ <nl> ▁ <add> ▁def ▁test _ 01 _ get _ product _ returns _ correct ly ( self ): ▁ <nl> ▁ <add> ▁obj ▁= ▁self . view . get _ object () ▁ <nl> ▁ <add> ▁inst ▁= ▁isinstance ( obj , product ) ▁ <nl> ▁ <add> ▁self . assertequal ( inst , ▁true ) ▁ <nl> ▁ <add> ▁ <nl> ▁ <add> ▁# ▁this ▁is ▁comment ed ▁out ▁until ▁the ▁following ▁gets ▁sol ved : ▁ <nl> ▁ <add> ▁# ▁ <url> ▁ <nl> ▁ <add> ▁# ▁def ▁test _0 2_ get _ templates _ return _ expected _ values ( self ): ▁ <nl> ▁ <add> ▁# ▁self . view ▁= ▁product detail view () ▁ <nl> ▁ <add> ▁# ▁tmp ▁= ▁self . view . get _ template _ names () ▁ <nl> ▁ <add> ▁# ▁import ▁ip db ; ▁ip db . set _ trace () ▁ <nl> ▁ <add> ▁# ▁self . assertequal ( len ( tmp ), ▁1) ▁ <nl> ▁ <nl>"

In [69]:
len(toks.split(" "))

502

In [74]:
(1e-5 > 5e-5)

False

In [79]:
with open("/workspace/iwslt14.tokenized.de-en/train.en") as file:
    lines = file.readlines()
    max_len = 0
    for line in lines:
        line_len = len(line.split(" "))
        if line_len > max_len:
            max_len = line_len

In [80]:
max_len

228

In [81]:
arg = "0.8, 0.15, 0.05"

In [85]:
[float(arg.strip()) for arg in arg.split(",")]

[0.8, 0.15, 0.05]

In [86]:
arg = '.diff, .msg'

In [87]:
arg.split(",")

['.diff', ' .msg']